In [0]:
%pip install yfinance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 143.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=19407041bef0383bdd1028a7b3bd5cf8dfb0a62315bf1640d967c78f4aefc022
  Stored in directory: /home/spark-b82b0bbf-3dc2-4d5f-aeac-6d/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.3-cp312-cp312-linux_aarch64.whl size=909445 sha256=6db08115

In [0]:
%pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
"""
Stock Portfolio Analysis Script
Fetches current stock prices and calculates portfolio performance
Outputs results to an Excel file
"""

import pandas as pd
import yfinance as yf
from datetime import datetime


In [0]:
def get_stock_data(ticker):
    """Fetch current stock price and basic info"""
    try:
        stock = yf.Ticker(ticker)
        current_price = stock.info.get("currentPrice") or stock.info.get(
            "regularMarketPrice"
        )
        if current_price is None:
            # Try getting from history if info doesn't have price
            hist = stock.history(period="1d")
            if not hist.empty:
                current_price = hist["Close"].iloc[-1]
        return current_price
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None


In [0]:
def export_to_excel(results, filename=None):
    """Export portfolio analysis to Excel file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"portfolio_analysis_{timestamp}.xlsx"

    df = pd.DataFrame(results)

    # Sort by Profit/Loss (descending - best performers first)
    # Handle 'N/A' values by treating them as 0 for sorting
    df_sorted = df.copy()
    df_sorted["PL_Sort"] = df_sorted["Profit/Loss"].apply(
        lambda x: x if x != "N/A" else -999999
    )
    df_sorted = df_sorted.sort_values("PL_Sort", ascending=False)
    df_sorted = df_sorted.drop("PL_Sort", axis=1)
    df = df_sorted

    # Calculate totals
    total_initial = df[df["Initial Value"] != "N/A"]["Initial Value"].sum()
    total_current = df[df["Current Value"] != "N/A"]["Current Value"].sum()
    total_pl = df[df["Profit/Loss"] != "N/A"]["Profit/Loss"].sum()
    total_return_pct = (
        ((total_current - total_initial) / total_initial) * 100
        if total_initial > 0
        else 0
    )

    # Add summary row
    summary = pd.DataFrame(
        [
            {
                "Ticker": "TOTAL",
                "Shares": "",
                "Purchase Price": "",
                "Current Price": "",
                "Initial Value": round(total_initial, 2),
                "Current Value": round(total_current, 2),
                "Profit/Loss": round(total_pl, 2),
                "Return %": round(total_return_pct, 2),
            }
        ]
    )

    df = pd.concat([df, summary], ignore_index=True)

    # Create Excel writer with formatting
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="Portfolio", index=False)

        # Get the worksheet
        worksheet = writer.sheets["Portfolio"]

        # Adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = min(max_length + 2, 20)
            worksheet.column_dimensions[column_letter].width = adjusted_width

        # Apply formatting to header
        from openpyxl.styles import Font, PatternFill, Alignment

        header_fill = PatternFill(
            start_color="366092", end_color="366092", fill_type="solid"
        )
        header_font = Font(bold=True, color="FFFFFF")

        for cell in worksheet[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = Alignment(horizontal="center")

        # Format summary row
        last_row = worksheet.max_row
        summary_fill = PatternFill(
            start_color="D9E1F2", end_color="D9E1F2", fill_type="solid"
        )
        summary_font = Font(bold=True)

        for cell in worksheet[last_row]:
            cell.fill = summary_fill
            cell.font = summary_font

    print(f"\n✓ Portfolio analysis exported to: {filename}")
    return filename, df

In [0]:
def analyze_portfolio(portfolio_data):
    """
    Analyze portfolio performance

    Parameters:
    portfolio_data: list of dicts with keys: 'ticker', 'shares', 'purchase_price'
    """
    results = []

    for holding in portfolio_data:
        ticker = holding["ticker"]
        shares = holding["shares"]
        purchase_price = holding["purchase_price"]

        print(f"Fetching data for {ticker}...")
        current_price = get_stock_data(ticker)

        if current_price:
            initial_value = shares * purchase_price
            current_value = shares * current_price
            profit_loss = current_value - initial_value
            profit_loss_pct = ((current_price - purchase_price) / purchase_price) * 100

            results.append(
                {
                    "Ticker": ticker,
                    "Shares": shares,
                    "Purchase Price": purchase_price,
                    "Current Price": round(current_price, 2),
                    "Initial Value": round(initial_value, 2),
                    "Current Value": round(current_value, 2),
                    "Profit/Loss": round(profit_loss, 2),
                    "Return %": round(profit_loss_pct, 2),
                }
            )
        else:
            results.append(
                {
                    "Ticker": ticker,
                    "Shares": shares,
                    "Purchase Price": purchase_price,
                    "Current Price": "N/A",
                    "Initial Value": round(shares * purchase_price, 2),
                    "Current Value": "N/A",
                    "Profit/Loss": "N/A",
                    "Return %": "N/A",
                }
            )

    return results

In [0]:
# EXAMPLE PORTFOLIO - Replace with your actual holdings
from collections import defaultdict

# Aggregate holdings by ticker: sum shares, average purchase price
agg_portfolio = defaultdict(lambda: {"shares": 0, "purchase_prices": []})

for holding in [
    {"ticker": "AMPY", "shares": 133, "purchase_price": 5.49},
    {"ticker": "HIMS", "shares": 36.39, "purchase_price": 49.04},
    {"ticker": "ITGR", "shares": 20, "purchase_price": 68.85},
    {"ticker": "LULU", "shares": 9, "purchase_price": 175.83},
    {"ticker": "MA", "shares": 2, "purchase_price": 558.50},
    {"ticker": "META", "shares": 2, "purchase_price": 664.25},
    {"ticker": "REGN", "shares": 3, "purchase_price": 570.33},
    {"ticker": "SMCI", "shares": 11, "purchase_price": 46.28},
    {"ticker": "UNH", "shares": 4, "purchase_price": 319.76},
    {"ticker": "V", "shares": 4, "purchase_price": 343.19},
    {"ticker": "VUSA.L", "shares": 5, "purchase_price": 84.69},
    {"ticker": "ZIM", "shares": 15, "purchase_price": 17.63},
    {"ticker": "AMPY", "shares": 72, "purchase_price": 6.23},
    {"ticker": "UNH", "shares": 3, "purchase_price": 300.33},
    {"ticker": "VUAG.L", "shares": 2, "purchase_price": 87.37},
    {"ticker": "VUSA.L", "shares": 2, "purchase_price": 83.97},
]:
    ticker = holding["ticker"]
    agg_portfolio[ticker]["shares"] += holding["shares"]
    agg_portfolio[ticker]["purchase_prices"].append(holding["purchase_price"])

portfolio = [
    {
        "ticker": ticker,
        "shares": agg_portfolio[ticker]["shares"],
        "purchase_price": sum(agg_portfolio[ticker]["purchase_prices"]) / len(agg_portfolio[ticker]["purchase_prices"]),
    }
    for ticker in agg_portfolio
]

print("=" * 60)
print("STOCK PORTFOLIO ANALYSIS")
print("=" * 60)
print(f"\nAnalyzing {len(portfolio)} holdings...\n")

# Analyze portfolio
results = analyze_portfolio(portfolio) # list

STOCK PORTFOLIO ANALYSIS

Analyzing 13 holdings...

Fetching data for AMPY...
Fetching data for HIMS...
Fetching data for ITGR...
Fetching data for LULU...
Fetching data for MA...
Fetching data for META...
Fetching data for REGN...
Fetching data for SMCI...
Fetching data for UNH...
Fetching data for V...
Fetching data for VUSA.L...
Fetching data for ZIM...
Fetching data for VUAG.L...


In [0]:
df_agg = pd.DataFrame(results)
display(df_agg)

Ticker,Shares,Purchase Price,Current Price,Initial Value,Current Value,Profit/Loss,Return %
AMPY,205.0,5.86,5.5,1201.3,1127.5,-73.8,-6.14
HIMS,36.39,49.04,36.26,1784.57,1319.5,-465.06,-26.06
ITGR,20.0,68.85,67.75,1377.0,1355.0,-22.0,-1.6
LULU,9.0,175.83,163.97,1582.47,1475.73,-106.74,-6.75
MA,2.0,558.5,527.58,1117.0,1055.16,-61.84,-5.54
META,2.0,664.25,597.69,1328.5,1195.38,-133.12,-10.02
REGN,3.0,570.33,725.34,1710.99,2176.02,465.03,27.18
SMCI,11.0,46.28,34.9,509.08,383.9,-125.18,-24.59
UNH,7.0,310.04499999999996,313.58,2170.31,2195.06,24.75,1.14
V,4.0,343.19,321.18,1372.76,1284.72,-88.04,-6.41


In [0]:
df_agg["PL_Sort"] = df_agg["Profit/Loss"].apply(lambda x: x if x != "N/A" else -999999)
df_agg = df_agg.sort_values("PL_Sort", ascending=False).drop("PL_Sort", axis=1)
display(df_agg)

Ticker,Shares,Purchase Price,Current Price,Initial Value,Current Value,Profit/Loss,Return %
REGN,3.0,570.33,725.34,1710.99,2176.02,465.03,27.18
VUSA.L,7.0,84.33,95.69,590.31,669.85,79.54,13.47
UNH,7.0,310.04499999999996,313.58,2170.31,2195.06,24.75,1.14
VUAG.L,2.0,87.37,96.82,174.74,193.64,18.9,10.82
ZIM,15.0,17.63,16.99,264.45,254.85,-9.6,-3.63
ITGR,20.0,68.85,67.75,1377.0,1355.0,-22.0,-1.6
MA,2.0,558.5,527.58,1117.0,1055.16,-61.84,-5.54
AMPY,205.0,5.86,5.5,1201.3,1127.5,-73.8,-6.14
V,4.0,343.19,321.18,1372.76,1284.72,-88.04,-6.41
LULU,9.0,175.83,163.97,1582.47,1475.73,-106.74,-6.75


In [0]:
# Calculate totals for columns from 'Purchase Price' to 'Return %'
totals_row = {
    "Ticker": "TOTALS",
    "Shares": round(df_agg[df_agg["Shares"] != "N/A"]["Shares"].sum(), 2),
    "Purchase Price": "N/A",
    "Current Price": "N/A",
    "Initial Value": round(df_agg[df_agg["Initial Value"] != "N/A"]["Initial Value"].sum(), 2),
    "Current Value": round(df_agg[df_agg["Current Value"] != "N/A"]["Current Value"].sum(), 2),
    "Profit/Loss": round(df_agg[df_agg["Profit/Loss"] != "N/A"]["Profit/Loss"].sum(), 2),
    "Return %": round(df_agg[df_agg["Return %"] != "N/A"]["Return %"].sum(), 2),
}

df_totals = pd.DataFrame([totals_row])
# df_agg_with_totals = pd.concat([df_agg, df_totals], ignore_index=True)
display(df_totals)

Ticker,Shares,Purchase Price,Current Price,Initial Value,Current Value,Profit/Loss,Return %
TOTALS,323.39,N/A,N/A,15183.48,14686.31,-497.16,-38.13


In [0]:
# export_to_excel(df_agg)


✓ Portfolio analysis exported to: portfolio_analysis_20251119_090941.xlsx


('portfolio_analysis_20251119_090941.xlsx',
     Ticker Shares Purchase Price  ... Current Value  Profit/Loss  Return %
 0     REGN    3.0         570.33  ...       2176.02       465.03    27.180
 1   VUSA.L    7.0          84.33  ...        669.92        78.53    13.485
 2   VUAG.L    2.0          87.37  ...        193.66        18.92    10.830
 3      UNH    7.0        310.045  ...       2195.06        15.03     1.240
 4      ZIM   15.0          17.63  ...        254.85        -9.60    -3.630
 5     ITGR   20.0          68.85  ...       1355.00       -22.00    -1.600
 6     AMPY  205.0           5.86  ...       1127.50       -51.23    -5.770
 7       MA    2.0          558.5  ...       1055.16       -61.84    -5.540
 8        V    4.0         343.19  ...       1284.72       -88.04    -6.410
 9     LULU    9.0         175.83  ...       1475.73      -106.74    -6.750
 10    SMCI   11.0          46.28  ...        383.90      -125.18   -24.590
 11    META    2.0         664.25  ...      

In [0]:
df_totals.head(2)

,Ticker,Shares,Purchase Price,Current Price,Initial Value,Current Value,Profit/Loss,Return %
0,TOTALS,323.39,N/A,N/A,15183.48,14686.31,-497.16,-38.13
